In [2]:
import numpy as np
from scipy import optimize

In [3]:
import matplotlib.pyplot as plt

<h2> Direct SLSQP </h2>

In [4]:
def Cost(x):
  return x[0]+x[1]

def Cost(x):
  return -x[1]

def Con0(x):
  return 1-x[0]**2-x[1]**2

def Con1(x):
  return x[1]

cons = [{'type': 'ineq', 'fun': Con0, 'ctol': 1e-4}, {'type': 'ineq', 'fun': Con1, 'ctol': 1e-4}]

In [5]:
optimize.minimize(Cost, [1,1], constraints=cons, method='SLSQP')

     fun: -1.0000000226284156
     jac: array([ 0., -1.])
 message: 'Optimization terminated successfully'
    nfev: 18
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([6.17040265e-07, 1.00000002e+00])

In [6]:
x0 = [1,1]
x = x0

In [7]:
sol = optimize.minimize(Cost, x, constraints=cons, options={'maxiter':2})
x = sol['x']
print(sol)
print(x)

     fun: -1.2500000018626451
     jac: array([ 0., -1.])
 message: 'Iteration limit reached'
    nfev: 6
     nit: 2
    njev: 2
  status: 9
 success: False
       x: array([0.25, 1.25])
[0.25 1.25]


<h2> Multiplier method</h2>

In [47]:
def Cost1(x):
  return x[0]+x[1]

def Cost2(x):
  return -x[1]

Cost = Cost1

def Con0(x):
  return 1-x[0]**2-x[1]**2

def Con1(x):
  return x[1]

cons = [{'type': 'ineq', 'fun': Con0, 'ctol': 1e-1}, {'type': 'ineq', 'fun': Con1, 'ctol': 1e-1}]

def ALagrangian(f, x, s, l, mu):
  c = [con['fun'](x) for con in cons]
  return f(x) + sum([-l[i]*(c[i]-s[i]) + mu/2*(c[i]-s[i])**2 for i in range(len(cons))])

In [48]:
eta_star = 1e-3
omega_star = 1e-6

x0 = [0,0]; x=x0
s0 = [con['fun'](x) for con in cons]; s=s0
l0 = [0,0]; l=l0



mu_list = [10]
omega_list = [1/mu_list[0]]
eta_list = [1/mu_list[0]**0.1]

v = x+s
bounds = [(None,None), (None,None), (0, None), (0,None)]



In [49]:
for _ in range(5):
  def AL_tmp(v):
    x, s = v[:len(v)//2], v[len(v)//2:]
    return ALagrangian(Cost, x,s,l,mu_list[-1])
  
  maxiter = 20
  sol = optimize.minimize(AL_tmp, v, bounds=bounds, 
                          options={'maxiter': maxiter,
                                   'ftol': omega_star,
                                   'gtol': 1e-6
                                  }, 
                          method="L-BFGS-B")
  v = sol['x']
  x, s = v[:len(v)//2], v[len(v)//2:]
  c = [con['fun'](x) for con in cons]

  print(sol)
  print(f'cost = {Cost(x)}')
  print(f'x = {x}')
  print(f's = {s}')
  print(f'c = {c}')
  print(f'l = {l}')
  print(f'mu = {mu_list[-1]}')
  
  if abs(max([(c[i]-s[i])/cons[i]['ctol'] for i in range(len(cons))])) < eta_list[-1]:
    if abs(max([(c[i]-s[i])/cons[i]['ctol'] for i in range(len(cons))])) < eta_star and sol['success']:
      print("optimization sucess!")
      break
    else:
      for i in range(len(cons)):
        l[i] = l[i] - mu_list[-1]*c[i]
      mu_list.append(mu_list[-1])
      eta_list.append(eta_list[-1]/mu_list[-1]**0.9)
      omega_list.append(omega_list[-1]/mu_list[-1])
  else:
    l = l
    mu_list.append(100*mu_list[-1])
    eta_list.append(1/mu_list[-1]**0.1)
    omega_list.append(1/mu_list[-1])
    
  print("optimization in progress...")
      
    

  

      fun: -1.0577452962315177
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00965037, 0.00097398, 0.48549884, 0.91060648])
  message: 'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 160
      nit: 20
     njev: 32
   status: 1
  success: False
        x: array([-1.01993031, -0.09106064,  0.        ,  0.        ])
cost = -1.1109909521146126
x = [-1.01993031 -0.09106064]
s = [0. 0.]
c = [-0.04854987780597589, -0.09106064211086927]
l = [0, 0]
mu = 10
optimization in progress...
      fun: -1.0003735177096662
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.83408845e-05,  4.44089210e-06,  4.99671837e-01,  9.91874205e-01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 40
      nit: 6
     njev: 8
   status: 0
  success: True
        x: array([-1.0006754 , -0.00812677,  0.        ,  0.        ])
cost = -1.0088021748303886
x = [-1.0006754  -0.00812677]
s = [0. 0.]
c = [-0.0014173014139658802, -0.0081

In [11]:
x

array([-1.00000010e+00, -5.17660744e-09])

In [14]:
s

[0, 0]

In [25]:
a = []
a.append(1)

In [26]:
a

[1]